In [17]:
import pandas as pd
import datetime
import sys

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
# OddsFeatures.py




In [18]:
import os
print(os.getcwd())
os.chdir('/Users/yannickflores/Perso Project/foreshadownba/foreshadwownba-ml/')
print(os.getcwd())

/Users/yannickflores/Perso Project/foreshadownba/foreshadwownba-ml
/Users/yannickflores/Perso Project/foreshadownba/foreshadwownba-ml


In [ ]:
# Models/prod_v2_w_logs_allmodels_w_featurewiz_2003_2022/ModelMetrics/FinalDFWithOdds.csv

In [23]:
test = pd.read_csv('./Models/prod_v2_w_logs_allmodels_w_featurewiz_2003_2022/ModelMetrics/FinalDFWithOdds.csv', sep=',')
test.head()

,id,game_date,tm_dom,tm_ext,extdom_dom,y_true_dom,model_type_dom,model_type_ext,model_pred_W_dom,model_pred_W_ext,model_pred_L_dom,model_pred_L_ext,MetaModel_pred_W_dom,MetaModel_pred_W_ext,MetaModel_pred_L_dom,MetaModel_pred_L_ext,model_predictions_dom,model_predictions_ext,BusinessRulesPred,team_ext,team_dom,odds_ext,odds_dom,score_team_ext,score_team_dom
0,2021-10-25_DET_ATL,2021-10-25,DET,ATL,dom,W,TestDataset2LinePerGamesProbWL_StartSeason,TestDataset2LinePerGamesProbWL_StartSeason,0.734179,0.263920,0.265821,0.736080,0.880297,0.110404,0.119703,0.889596,W,L,W,ATL,DET,1.15,5.87,122.0,104.0
1,2021-10-25_DET_ATL,2021-10-25,DET,ATL,dom,W,TestDataset2LinePerGamesProbWL_StartSeason,TestDataset2LinePerGamesProbWL_StartSeason,0.734179,0.263920,0.265821,0.736080,0.880297,0.110404,0.119703,0.889596,W,L,W,ATL,DET,1.15,5.87,122.0,104.0
2,2021-10-22_TOR_BOS,2021-10-22,TOR,BOS,dom,L,TestDataset2LinePerGamesProbWL_StartSeason,TestDataset2LinePerGamesProbWL_StartSeason,0.538976,0.375637,0.461024,0.624363,0.829583,0.137097,0.170417,0.862903,W,L,W,BOS,TOR,1.35,3.32,83.0,115.0
3,2021-10-22_TOR_BOS,2021-10-22,TOR,BOS,dom,L,TestDataset2LinePerGamesProbWL_StartSeason,TestDataset2LinePerGamesProbWL_StartSeason,0.538976,0.375637,0.461024,0.624363,0.829583,0.137097,0.170417,0.862903,W,L,W,BOS,TOR,1.35,3.32,83.0,115.0
4,2021-10-27_WAS_BOS,2021-10-27,WAS,BOS,dom,L,TestDataset2LinePerGamesProbWL_StartSeason,TestDataset2LinePerGamesProbWL_StartSeason,0.555450,0.453883,0.444550,0.546117,0.612642,0.399645,0.387358,0.600355,W,L,W,BOS,WAS,1.57,2.51,107.0,116.0


In [24]:
test.shape

(7468, 25)

In [25]:
test = test.drop_duplicates(subset=(['id', 'tm_dom', 'tm_ext']))
test.shape

(3734, 25)

In [26]:
odds_df = pd.read_csv('./odds/odds_2017_2022.csv', sep=';')

In [20]:
odds_df.shape

(6612, 6)

In [27]:
odds_df.head()

,date,team_a,team_b,score_a_b,odds_a,odds_b
0,09/06/2018,CLE,GSW,85:108,2.44,1.61
1,07/06/2018,CLE,GSW,102:110,2.28,1.68
2,04/06/2018,GSW,CLE,122:103,1.19,5.33
3,01/06/2018,GSW,CLE,124:114 OT,1.12,7.16
4,29/05/2018,HOU,GSW,92:101,3.31,1.37


In [15]:

#----------------------------------------
# Convert date to dateformat
odds_df['date'] = pd.to_datetime(odds_df['date'], format='%d/%m/%Y')

#----------------------------------------
#remove row where odds = -
odds_df = odds_df[odds_df['odds_a']!= '-']

#----------------------------------------
# Convert odds to numeric
odds_df['odds_a'] = pd.to_numeric(odds_df['odds_a'])
odds_df['odds_b'] = pd.to_numeric(odds_df['odds_b'])

score = odds_df['score_a_b'].str.split(":", expand=True)

score.columns = [
'score_team_a',
'score_team_b']

score = pd.DataFrame(score)

score["score_team_b"] = score["score_team_b"].str.replace("[OT]", "")
score["score_team_b"] = score["score_team_b"].str.extract('(\d+)')

score['score_team_a'] = pd.to_numeric(score['score_team_a'])
score['score_team_b'] = pd.to_numeric(score['score_team_b'])

odds_df = pd.concat(
[odds_df, score],
axis = 1
)

odds_df = odds_df.drop(['score_a_b'], axis=1)   

#----------------------------------------
# Create anc append another data frame where date = date-1 day
# We do that because we have french date for a game
odds_df_2 = odds_df
odds_df_2['date'] = odds_df_2['date'] - pd.Timedelta(days=1)
OddsFinal = pd.concat([odds_df, odds_df_2], axis = 0)

#----------------------------------------
# Columns with _a coresponds to Team playing ext
OddsFinal.columns = ['game_date', 'team_ext', 'team_dom', 'odds_ext', 'odds_dom', 'score_team_ext', 'score_team_dom']
OddsFinal.head()


/var/folders/3x/hm8hmk5d5v71tnm88n9rfb8c0000gn/T/ipykernel_18346/1635654083.py:22: FutureWarning: The default value of regex will change from True to False in a future version.
  score["score_team_b"] = score["score_team_b"].str.replace("[OT]", "")


,game_date,team_ext,team_dom,odds_ext,odds_dom,score_team_ext,score_team_dom
0,2018-06-08,CLE,GSW,2.44,1.61,85,108
1,2018-06-06,CLE,GSW,2.28,1.68,102,110
2,2018-06-03,GSW,CLE,1.19,5.33,122,103
3,2018-05-31,GSW,CLE,1.12,7.16,124,114
4,2018-05-28,HOU,GSW,3.31,1.37,92,101


In [16]:
OddsFinal.shape

(13220, 7)

In [ ]:

#----------------------------------------
# Filter Business rules models and only get the season from 2017 - 2022
FinalDFWithouthOdds = FinalDFWithouthOdds[pd.to_datetime(FinalDFWithouthOdds['game_date']) > pd.to_datetime('2017-09-01')]

# print(FinalDFWithouthOdds[['game_date', 'tm_dom', 'tm_ext', 'extdom_dom']][FinalDFWithouthOdds['game_date']=='2018-03-18'])
# print(OddsFinal[OddsFinal['game_date'].isin(['2018-03-18', '2018-03-17'])])


#----------------------------------------
# Cast date columns as string
OddsFinal['game_date'] = OddsFinal['game_date'].astype(str)
FinalDFWithouthOdds['game_date'] = FinalDFWithouthOdds['game_date'].astype(str)

#----------------------------------------
# Left Join to add the datesd to the Business rules final mdoels
FinalDFWithOdds =  pd.merge(
FinalDFWithouthOdds,
OddsFinal,
how='left',
left_on=['game_date', 'tm_dom', 'tm_ext'],
right_on=['game_date', 'team_dom', 'team_ext'])

#----------------------------------------
# Remove Nas values: Come from missing odds and date issue
FinalDFWithOdds = FinalDFWithOdds.dropna()